In [4]:
!pip install streamlit youtube-transcript-api textblob wordcloud sumy


  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 56.5 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21691 sha256=6b1117e4215a79b2b693cc3bf7311c6d49f527099ad83110ecc5f75a6e386c5e
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13704 sha256=c747cdec277c9ba137765a5a94160f08c51a000184254876f08fc836eab532cf
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built breadability docopt


In [5]:
import os
import streamlit as st
from youtube_transcript_api import YouTubeTranscriptApi
from textblob import TextBlob
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer

# Function to extract transcript and language code
def get_transcript(youtube_url):
    video_id = youtube_url.split("v=")[-1]
    transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)

    try:
        transcript = transcript_list.find_manually_created_transcript()
        language_code = transcript.language_code
    except:
        try:
            generated_transcripts = [trans for trans in transcript_list if trans.is_generated]
            transcript = generated_transcripts[0]
            language_code = transcript.language_code
        except:
            raise Exception("No suitable transcript found.")

    full_transcript = " ".join([part['text'] for part in transcript.fetch()])
    return full_transcript, language_code

# Sentiment Analysis on the transcript
def perform_sentiment_analysis(text):
    blob = TextBlob(text)
    sentiment_score = blob.sentiment.polarity  # -1 to 1 (negative to positive)
    return sentiment_score

# Keyword Extraction using frequency-based analysis
def extract_keywords(text, num_keywords=10):
    words = text.split()
    word_counts = Counter(words)
    common_words = word_counts.most_common(num_keywords)
    return common_words

# Generate Word Cloud
def generate_wordcloud(text):
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    st.pyplot(plt)

# Generate Bar Plot for Keywords
def plot_keywords(keywords):
    words, counts = zip(*keywords)
    sns.barplot(x=list(counts), y=list(words))
    plt.xlabel('Frequency')
    plt.ylabel('Keywords')
    plt.title('Top Keywords')
    st.pyplot(plt)

# Summarize the transcript using Sumy LSA Summarizer
def summarize_with_sumy(transcript):
    parser = PlaintextParser.from_string(transcript, Tokenizer("english"))
    summarizer = LsaSummarizer()
    summary = summarizer(parser.document, 5)  # 5 sentences in the summary

    return "\n".join([str(sentence) for sentence in summary])

# Calculate the content score based on sentiment and keywords
def calculate_content_score(sentiment_score, keywords):
    keyword_density = sum([count for _, count in keywords]) / len(keywords)
    score = (sentiment_score + 1) * 50 + keyword_density * 10  # Adjust weightings as needed

    return max(0, round(score)) # Clamp score between 0 and 100

# Main Streamlit App
def main():
    st.title('📹 YouTube Video Content Analyzer & Summarizer')
    link = st.text_input('Enter the YouTube video URL:')

    if st.button('Analyze Video'):
        if link:
            try:
                progress = st.progress(0)
                status_text = st.empty()

                status_text.text('Fetching the transcript...')
                progress.progress(25)

                # Extract transcript and language code
                transcript, language_code = get_transcript(link)

                # Perform sentiment analysis
                sentiment_score = perform_sentiment_analysis(transcript)
                st.write(f"Sentiment Score: {sentiment_score:.2f} (Range: -1 to 1)")

                # Extract keywords
                keywords = extract_keywords(transcript)
                st.write("*Top Keywords:*")
                st.write(keywords)

                # Visualize Word Cloud and Keyword Bar Plot
                st.write("### Word Cloud")
                generate_wordcloud(transcript)

                st.write("### Keyword Frequency Plot")
                plot_keywords(keywords)

                progress.progress(75)
                status_text.text('Generating summary...')

                # Generate summary using Sumy
                summary = summarize_with_sumy(transcript)
                st.markdown("### Summary:")
                st.markdown(summary)

                # Calculate content score
                content_score = calculate_content_score(sentiment_score, keywords)
                st.write(f"### Content Score: {content_score}/100")
                st.progress(content_score)

                status_text.text('Analysis Complete.')
                progress.progress(100)

            except Exception as e:
                st.error(f"Error: {str(e)}")
        else:
            st.warning('Please enter a valid YouTube link.')

if __name__ == "__main__":
    main()

2024-10-15 16:40:35.396 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-15 16:40:35.799 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-10-15 16:40:35.806 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-15 16:40:35.811 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-15 16:40:35.813 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-15 16:40:35.821 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-15 16:40:35.832 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-15 16:40:35.840 Session state does not 

In [ ]:
!streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.148.243.95:8501

